In [1]:
from tensorflow.keras.layers import (
    Dense, LSTM, Convolution1D,
    Dropout, MaxPooling1D, BatchNormalization
)
from tensorflow.keras.layers import (
    Flatten, Input, TimeDistributed,
    Activation, RepeatVector, Permute,
    Lambda, Concatenate, Bidirectional, 
    Masking, concatenate, multiply
)
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential, Model

class ModelZoo(object):
    
    @staticmethod
    def plain_model(input_shape, hidden_1=600, hidden_2=400,
                    dropout_1=0.3, dropout_2=0.3, optimizer='adam',
                    number_of_roles=34):
        
        plain_model = Sequential()
        plain_model.add(Dense(hidden_1, 
                          #input_shape=(plain_features.shape[1],), 
                          input_shape = input_shape,
                          activation = tf.keras.activations.relu))
        plain_model.add(Dropout(dropout_1))
    
        plain_model.add(Dense(hidden_2))
        plain_model.add(BatchNormalization())
        plain_model.add(Activation('relu'))
        plain_model.add(Dropout(dropout_2))
    
        plain_model.add(Dense(number_of_roles))
        plain_model.add(BatchNormalization())
        plain_model.add(Activation('softmax'))
    
        plain_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
        return plain_model

    @staticmethod
    def sparse_model(categ_size, emb_size, 
                     hidden_plain=400, hidden_embed=100, 
                     hidden_2=400, dropout_1=0.3, dropout_2=0.3,
                     optimizer='adam', number_of_roles=34):
        
        input_plain = Input(shape=(categ_size,), name = 'input_categorical')
        input_pred_embed = Input(shape=(emb_size,), name = 'pred_embed')
        input_arg_embed = Input(shape=(emb_size,), name = 'arg_embed')
    
        plain = Dense(hidden_plain)(input_plain)
        plain = BatchNormalization()(plain)
        plain = Activation('relu')(plain)
    
        def embed_submodel(inpt):
            embed = Dense(hidden_embed)(inpt)
            embed = BatchNormalization()(embed)
            embed = Activation('relu')(embed)
            return embed
    
        embed_pred = embed_submodel(input_pred_embed)
        embed_arg = embed_submodel(input_arg_embed)
    
        final = Concatenate(axis = 1)([embed_pred, embed_arg, plain])
        final = Dropout(dropout_1)(final)
        final = Dense(hidden_2)(final)
        final = BatchNormalization()(final)
        final = Activation('relu')(final)
        final = Dropout(dropout_2)(final)
        final = Dense(number_of_roles)(final)
        final = BatchNormalization()(final)
        final = Activation('softmax')(final)
    
        model = Model([input_arg_embed, input_pred_embed, input_plain], final)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
        return model

In [45]:
class SparseModel(tf.keras.Model):
    def __init__(self, hidden_categorical=400, hidden_embeddings=100, hidden_final=400, dropout_1=0.3, dropout_2=0.3, number_of_roles=34):
        super(SparseModel, self).__init__(name="sparse_model")
        self.fc_categorical = nn.Dense(units=hidden_categorical)
        self.fc_pred_embedding = nn.Dense(units=hidden_embeddings)
        self.fc_arg_embedding = nn.Dense(units=hidden_embeddings)
        self.fc_concat = nn.Dense(units=hidden_final)
        self.fc_final = nn.Dense(units=number_of_roles)
        self.dropout1 = nn.Dropout(rate=dropout_1)
        self.dropout2 = nn.Dropout(rate=dropout_2)
        self.batch_norms = {f'bn_{i}': nn.BatchNormalization() for i in ['categorical', 'embed_pred', 'embed_arg', 'concat', 'final']}
        
    def call(self, inputs):
        embed_pred, embed_arg, categorical = inputs
        embed_pred = self.fc_pred_embedding(embed_pred)
        embed_pred = tf.nn.relu(self.batch_norms['bn_embed_pred'](embed_pred))
        
        embed_arg = self.fc_arg_embedding(embed_arg)
        embed_arg = tf.nn.relu(self.batch_norms['bn_embed_arg'](embed_arg))
        
        categorical = self.fc_categorical(categorical)
        categorical = tf.nn.relu(self.batch_norms['bn_categorical'](categorical))
        
        concat = tf.concat([embed_pred, embed_arg, categorical], axis=1)
        concat = self.dropout1(concat)
        concat = self.fc_concat(concat)
        concat = tf.nn.relu(self.batch_norms['bn_concat'](concat))
        
        final = self.dropout2(concat)
        final = self.fc_final(final)
        final = tf.nn.softmax(self.batch_norms['bn_final'](final))
        
        return final
        

In [35]:
import tensorflow as tf
from tensorflow.keras import layers as nn

In [36]:
class SimpleModel(tf.keras.Model):
    def __init__(self, hidden_1=600, hidden_2=400, dropout_1=0.3, dropout_2=0.3, number_of_roles=34):
        super(SimpleModel, self).__init__(name='simple_model')
        self.fc1 = nn.Dense(units=hidden_1, activation=tf.nn.relu)
        self.dropout1 = nn.Dropout(rate=dropout_1)
        self.dropout2 = nn.Dropout(rate=dropout_2)
        self.fc2 = nn.Dense(units=hidden_2)
        self.bn1 = nn.BatchNormalization()
        self.fc3 = nn.Dense(units=number_of_roles)
        self.bn2 = nn.BatchNormalization()
        
    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = tf.nn.relu(self.bn1(x))
        x = self.dropout2(x)
        x = self.bn2(self.fc3(x))
        return tf.nn.softmax(x)

In [37]:
tf.enable_eager_execution()

In [38]:
tf.executing_eagerly()

True

In [39]:
example = tf.random.normal(shape=(16, 700))

In [40]:
model = SimpleModel()

In [46]:
sparse = SparseModel()

In [42]:
tf.reduce_sum(model(example), axis=-1)

<tf.Tensor: id=771, shape=(16,), dtype=float32, numpy=
array([0.9999999 , 0.9999999 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.99999994, 1.        , 1.        , 1.        , 1.0000001 ,
       0.99999994], dtype=float32)>

In [47]:
embed_pred = tf.random.normal(shape=(16, 300))
embed_arg  = tf.random.normal(shape=(16, 300))
categoric  = tf.random.normal(shape=(16, 700))

In [52]:
list(tf.reduce_sum(sparse((embed_pred, embed_pred, categoric)), axis=1).numpy())

[0.9999999,
 1.0,
 1.0,
 0.99999994,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999,
 1.0,
 0.9999999,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999]